In [51]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import common.common_machine_learning as common
import common.feature_num as feature_num
import common.features_precio_promedio_metroscubiertos as feature_metroscubiertos
import common.features_precio_promedio_ciudad as feature_ciudades

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

import lightgbm as lgb

In [81]:
TARGET = "precio"
MAX_ITER = 200

### Funciones auxiliares

In [3]:
def limpiar(df):
    df.antiguedad = df.antiguedad.fillna(df.antiguedad.mean())
    df.metroscubiertos = df.metroscubiertos.fillna(df.metroscubiertos.mean())
    df.habitaciones = df.habitaciones.fillna(df.habitaciones.mean())
    df.garages = df.garages.fillna(0)
    df.banos = df.banos.fillna(1)
    df.tipodepropiedad = df.tipodepropiedad.fillna('Casa')
    df.metroscubiertos = df.metroscubiertos.fillna(df.metroscubiertos.mean())
    df.metrostotales = df.metrostotales.fillna(df.metrostotales.mean())
    df.gimnasio = df.gimnasio.fillna(0)
    df.usosmultiples = df.usosmultiples.fillna(0)
    df.piscina = df.piscina.fillna(0)
    df.escuelascercanas = df.escuelascercanas.fillna(0)
    df.centroscomercialescercanos = df.centroscomercialescercanos.fillna(0)
    df["metroscubiertos"] = df["metroscubiertos"].fillna(df["metroscubiertos"].mean())
    df.fillna(value = {"tipodepropiedad" : df["tipodepropiedad"].mode().to_string(),
                        "provincia" : df["provincia"].mode().to_string(),
                        "ciudad": df["ciudad"].mode().to_string()}, inplace = True)
    
def evaluar_rf(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Performance del modelo:')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [4]:
def nuevas_features(df, precios_tipo,precio_m2,default_m2):
    df['ratio_cubierto'] = df.apply(lambda x: x['metroscubiertos']/x['metrostotales'] if x['metrostotales'] else 1, axis = 1)
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: precios_tipo.loc[x]['precio_por_tipo'])
    df['precio_x_m2'] = df.apply(lambda x: precio_x_m2.get(x['ciudad'],default_m2), axis = 1)

# Preparo el dataset

## Carga de datos

In [5]:
train = pd.read_csv('sets_de_datos/train.csv', index_col = 0)
test = pd.read_csv('sets_de_datos/test.csv', index_col = 0)

In [6]:
train["fecha"] = pd.to_datetime(train["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')
test["fecha"] = pd.to_datetime(test["fecha"], format="%Y-%m-%d %H:%M:%S", errors='coerce')

## Limpio las columnas que voy a usar

In [7]:
train["provincia"].replace(["", np.nan], [train["provincia"].mode(), train["provincia"].mode()], inplace=True)
test["provincia"].replace(["", np.nan], [test["provincia"].mode(), test["provincia"].mode()], inplace=True)

In [8]:
limpiar(train)
limpiar(test)

In [9]:
test['tipodepropiedad'] = test['tipodepropiedad'].str.replace('0    Casa',"Casa")
train = train[train["tipodepropiedad"].isin(test["tipodepropiedad"].unique())]
train.dropna(subset=["tipodepropiedad"], inplace=True)

In [10]:
train.isna().sum()

titulo                          5386
descripcion                     1619
tipodepropiedad                    0
direccion                      53072
ciudad                             0
provincia                          0
antiguedad                         0
habitaciones                       0
garages                            0
banos                              0
metroscubiertos                    0
metrostotales                      0
idzona                         28621
lat                           123486
lng                           123486
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [11]:
test.isna().sum()

titulo                         1378
descripcion                     401
tipodepropiedad                   0
direccion                     13191
ciudad                            0
provincia                         0
antiguedad                        0
habitaciones                      0
garages                           0
banos                             0
metroscubiertos                   0
metrostotales                     0
idzona                         7179
lat                           30695
lng                           30695
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

## Cargo features adicionales

In [12]:
df_precios_por_tipo = train.groupby('tipodepropiedad').agg({'precio':'mean'}).rename(columns={'precio':'precio_por_tipo'})
train['precio_x_m2'] = train['precio']/train['metroscubiertos']
precio_x_m2 = train.groupby('ciudad').agg({'precio_x_m2':'mean'}).to_dict()['precio_x_m2']
default = train.groupby('ciudad').agg({'precio_x_m2':'mean'})['precio_x_m2'].mean()
promedios = train.set_index('ciudad')\
            .join(train.groupby('ciudad')\
                  .agg({'habitaciones':'mean', 'garages':'mean', 'banos':'mean'})\
                      .rename(columns={'habitaciones':'mean_hab', 'banos':'mean_ban', 'garages':'mean_gar'}))\
                        [['mean_hab','mean_gar','mean_ban']].to_dict()

In [13]:
nuevas_features(train, df_precios_por_tipo, precio_x_m2,default)

In [14]:
nuevas_features(test, df_precios_por_tipo, precio_x_m2,default)

In [15]:
#features_descripcion_train = pd.read_csv('data/dima_train_categorias_descripciones.csv', index_col = 0)
#train = train.join(features_descripcion_train)

In [16]:
#features_descripcion_test = pd.read_csv('data/dima_test_categorias_descripciones.csv', index_col = 0)
#test = test.join(features_descripcion_test)

In [17]:
train = feature_num.completar_lat_lng_con_provincias_y_ciudades(train)
train = feature_num.completar_lat_lng_con_idzona_mean(train)
feature_num.completar_lat_lng_con_promedio_Mexico(train)

In [18]:
test = feature_num.completar_lat_lng_con_provincias_y_ciudades(test)
test = feature_num.completar_lat_lng_con_idzona_mean(test)
feature_num.completar_lat_lng_con_promedio_Mexico(test)

In [19]:
train.isna().sum()

titulo                         5386
descripcion                    1619
tipodepropiedad                   0
direccion                     53072
ciudad                            0
provincia                         0
antiguedad                        0
habitaciones                      0
garages                           0
banos                             0
metroscubiertos                   0
metrostotales                     0
idzona                        28621
lat                               0
lng                               0
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
precio_x_m2                       0
ratio_cubierto                    0
dtype: int64

In [20]:
train = feature_ciudades.asignar_precio_promedio_por_cantidad_de_banos_por_ciudad(train)

In [21]:
train = feature_ciudades.asignar_precio_promedio_por_cantidad_de_habitaciones_por_ciudad(train)

In [22]:
train = feature_ciudades.asignar_precio_promedio_por_cantidad_de_garages_por_ciudad(train)

In [23]:
test = feature_ciudades.asignar_precio_promedio_por_cantidad_de_banos_por_ciudad(test)

In [24]:
test = feature_ciudades.asignar_precio_promedio_por_cantidad_de_habitaciones_por_ciudad(test)

In [25]:
test = feature_ciudades.asignar_precio_promedio_por_cantidad_de_garages_por_ciudad(test)

In [26]:
#train = feature_metroscubiertos.agregar_feature_precio_promedio_banos_por_metroscubiertos(train)

In [27]:
#train = feature_metroscubiertos.agregar_feature_precio_promedio_habitaciones_por_metroscubiertos(train)

In [28]:
#train = feature_metroscubiertos.agregar_feature_precio_promedio_garages_por_metroscubiertos(train)

In [29]:
#test = feature_metroscubiertos.agregar_feature_precio_promedio_banos_por_metroscubiertos(test)

In [30]:
#test = feature_metroscubiertos.agregar_feature_precio_promedio_habitaciones_por_metroscubiertos(test)

In [31]:
#test = feature_metroscubiertos.agregar_feature_precio_promedio_garages_por_metroscubiertos(test)

In [32]:
train.shape

(239998, 27)

In [33]:
test.shape

(60000, 26)

In [34]:
train.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'precio_x_m2', 'ratio_cubierto',
       'banos_preciopromedio_ciudad', 'habitaciones_preciopromedio_ciudad',
       'garages_preciopromedio_ciudad'],
      dtype='object')

## Me quedo con algunas features

In [39]:
columnas_train = ["provincia", "tipodepropiedad", "antiguedad", "lat", "lng", "gimnasio", "usosmultiples", "piscina",
                  "centroscomercialescercanos", "ratio_cubierto", "banos_preciopromedio_ciudad",
                  "habitaciones_preciopromedio_ciudad", "garages_preciopromedio_ciudad", "precio"]

columnas_test = ["provincia", "tipodepropiedad", "antiguedad", "lat", "lng", "gimnasio", "usosmultiples", "piscina",
                  "centroscomercialescercanos", "ratio_cubierto", "banos_preciopromedio_ciudad",
                  "habitaciones_preciopromedio_ciudad", "garages_preciopromedio_ciudad"]

In [40]:
train = train[columnas_train]
test = test[columnas_test]

In [41]:
train_OHE  = pd.get_dummies(train)

In [42]:
test_OHE = pd.get_dummies(test)

In [43]:
train_OHE.shape

(239998, 45)

In [44]:
test_OHE.shape

(60000, 44)

In [46]:
train_OHE.banos_preciopromedio_ciudad = train_OHE.banos_preciopromedio_ciudad.fillna(train_OHE.banos_preciopromedio_ciudad.mean())
train_OHE.habitaciones_preciopromedio_ciudad = train_OHE.habitaciones_preciopromedio_ciudad.fillna(train_OHE.habitaciones_preciopromedio_ciudad.mean())
train_OHE.garages_preciopromedio_ciudad = train_OHE.garages_preciopromedio_ciudad.fillna(train_OHE.garages_preciopromedio_ciudad.mean())

test_OHE.banos_preciopromedio_ciudad = test_OHE.banos_preciopromedio_ciudad.fillna(test_OHE.banos_preciopromedio_ciudad.mean())
test_OHE.habitaciones_preciopromedio_ciudad = test_OHE.habitaciones_preciopromedio_ciudad.fillna(test_OHE.habitaciones_preciopromedio_ciudad.mean())
test_OHE.garages_preciopromedio_ciudad = test_OHE.garages_preciopromedio_ciudad.fillna(test_OHE.garages_preciopromedio_ciudad.mean())


# Tuneo

### Divido el train

In [47]:
X = train_OHE.drop([TARGET], axis = 1).copy().values
y = list(train_OHE[TARGET].copy())

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 0)

### Grilla de parápametros

In [82]:
hiperparametros = {
    "learning_rate": [0.03, 0.01, 0.1, 0.3],
    "n_estimators": [100, 200, 250, 300, 350, 400],
    "max_depth": [15, 20, 25, 30, 40, 50, 60],
    "reg_alpha": [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    "reg_lambda": [0, 1e-1, 1, 5, 10, 20, 50, 100],
    "num_leaves": [5, 10, 15, 20, 25, 30],
    "min_child_weight": [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],

}

### Tuneo

In [83]:
modelo = lgb.LGBMRegressor(learning_rate = 0.5)

inicio = time.time()


modelo_random = RandomizedSearchCV(estimator = modelo, param_distributions = hiperparametros, n_iter = MAX_ITER, cv = 3, verbose=2, random_state=42, n_jobs = -1)

fin = time.time()

modelo_random.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 18.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.5, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_...
                                        'max_depth': [15, 20, 25, 30, 40, 50,
                                                      60],
                                        'mi

In [84]:
print("El tuneo tardó: {} minutos.".format((fin - inicio) / 60))

El tuneo tardó: 0.0 minutos.


In [85]:
print("Mejores parámetros:")
print(modelo_random.best_params_)

Mejores parámetros:
{'reg_lambda': 10, 'reg_alpha': 7, 'num_leaves': 30, 'n_estimators': 400, 'min_child_weight': 100.0, 'max_depth': 60, 'learning_rate': 0.3}


Mejores parámetros:
{'reg_lambda': 5, 'reg_alpha': 50, 'num_leaves': 30, 'n_estimators': 300, 'min_child_weight': 0.1, 'max_depth': 15, 'learning_rate': 0.3}

# Entrenamiento

In [86]:
mejores_hiperparametros = modelo_random.best_params_

In [87]:
mejor_lgb = lgb.LGBMRegressor(reg_lambda = mejores_hiperparametros["reg_lambda"],
                              num_leaves = mejores_hiperparametros["num_leaves"],
                              n_estimators = mejores_hiperparametros["n_estimators"],
                              min_child_weight = mejores_hiperparametros["min_child_weight"],
                              max_depth = mejores_hiperparametros["max_depth"],
                              learning_rate = mejores_hiperparametros["learning_rate"])
                                    
mejor_lgb.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.3, max_depth=60,
              min_child_samples=20, min_child_weight=100.0, min_split_gain=0.0,
              n_estimators=400, n_jobs=-1, num_leaves=30, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=10, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [88]:
evaluar_rf(mejor_lgb, X_test, y_test)

Performance del modelo:
Average Error: 787003.0393 degrees.
Accuracy = 59.12%.


59.119928022612115

# Predicción

In [90]:
y_pred = mejor_lgb.predict(test_OHE)

In [91]:
df = pd.DataFrame({"id": test.index, "target": y_pred})

In [92]:
filename = 'LightLGB_00.csv'

df.to_csv(filename,index=False)